In [6]:
!guardrails hub install hub://guardrails/two_words --quiet

Installing hub://guardrails/two_words...
✅Successfully installed guardrails/two_words!




## Input Validation

Guardrails supports validating inputs (prompts, instructions, msg_history) with string validators.

In XML, specify the validators on the `prompt` or `instructions` tag, as such:

In [7]:
from guardrails import Guard

rail_spec = """
<rail version="0.1">
<prompt
    validators="hub://guardrails/two_words"
    on-fail-two-words="exception"
>
This is not two words
</prompt>
<output type="string">
</output>
</rail>
"""

guard = Guard.from_rail_string(rail_spec)

When `fix` is specified as the on-fail handler, the prompt will automatically be amended before calling the LLM.

In any other case (for example, `exception`), a `ValidationError` will be returned in the outcome.

In [8]:
import openai
from guardrails.errors import ValidationError

try:
    guard(
        openai.chat.completions.create,
    )
except ValidationError as e:
    print(e)

When using pydantic to initialize a `Guard`, input validators can be specified by composition, as such:

In [9]:
from guardrails.hub import TwoWords
from pydantic import BaseModel


class Pet(BaseModel):
    name: str
    age: int


guard = Guard.from_pydantic(Pet)
guard.with_prompt_validation([TwoWords(on_fail="exception")])

try:
    guard(
        openai.chat.completions.create,
        prompt="This is not two words",
    )
except ValidationError as e:
    print(e)

/Users/zaydsimjee/workspace/guardrails/docs/examples/.venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3577: FutureWarning: The `with_prompt_validation` method is deprecated,
        and will be removed in 0.5.x. Instead, please use
        `Guard().use(YourValidator, on='prompt')`.
  exec(code_obj, self.user_global_ns, self.user_ns)


Validation failed for field with errors: Value must be exactly two words
